In [ ]:
!pip install transformers datasets accelerate rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
import os
import json
import torch
import numpy as np

from torch.utils.data import DataLoader

from tqdm import tqdm
from rouge import Rouge
from datasets import Dataset, load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, pipeline, Seq2SeqTrainingArguments, Seq2SeqTrainer

os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## **1. 모델 및 토크나이저 정의**

In [ ]:
MODEL_CHECKPOINT = 'gogamza/kobart-base-v2'

TRAIN_TEST_SPLIT = 0.2
MAX_INPUT_LENGTH = 512      # encoder에 들어갈 max input 길이
MIN_TARGET_LENGTH = 5       # decoder에 들어갈 min input 길이
MAX_TARGET_LENGTH = 128     # decoder에 들어갈 max input 길이
BATCH_SIZE = 8              # 모델 학습에 사용할 batch size 크기
LEARNING_RATE = 3e-5        # 모델 학습에 사용할 learning rate
MAX_EPOCHS = 5              # 모델 학습에 사용할 epoch 수

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT).to(device)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


## **2. 데이터셋 다운로드**

In [ ]:
!gdown 1S5kUCc-u-F2w5JOgS81w2ht6Wmcjo7-y

Downloading...
From: https://drive.google.com/uc?id=1S5kUCc-u-F2w5JOgS81w2ht6Wmcjo7-y
To: /content/sample.jsonl
100% 8.64M/8.64M [00:00<00:00, 88.2MB/s]


In [ ]:
!ls

sample_data  sample.jsonl


In [ ]:
!apt-get install jq
!head -n 1 ./sample.jsonl | jq '.'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libjq1 libonig5
The following NEW packages will be installed:
  jq libjq1 libonig5
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 313 kB of archives.
After this operation, 1,062 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libonig5 amd64 6.9.4-1 [142 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 libjq1 amd64 1.6-1ubuntu0.20.04.1 [121 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 jq amd64 1.6-1ubuntu0.20.04.1 [50.2 kB]
Fetched 313 kB in 1s (529 kB/s)
Selecting previously unselected package libonig5:amd64.
(Reading database ... 122520 files and directories currently installed.)
Preparing to unpack .../libonig5_6.9.4-1_amd64.deb ...
Unpacking libonig5:amd64 (6.9.4-1) ...
Selecting previously unselected pa

In [ ]:
dataset_path = './sample.jsonl'

document, summary = [], []

with open(dataset_path, 'r', encoding='utf-8') as f:
  while True:
    line = f.readline()

    if not line:
      break
    
    doc = json.loads(line)

    document.append(' '.join(doc['article_original']))
    summary.append(doc['abstractive'])

raw_datasets = Dataset.from_dict({'document': document, 'summary': summary})

In [ ]:
raw_datasets[0] # 원본 데이터 형식 및 내용 확인

{'document': '보조금 집행 위법행위·지적사례 늘어 특별감사반, 2017~2018년 축제 점검 충주시가 민간에게 지원되는 보조사업의 대형축제와 관련해 선정·집행·정산 등 운영실태 전반에 대한 자체 감사를 실시할 계획이라고 밝혔다. 이는 최근 민간보조사업의 증가와 더불어 보조금 집행관리에 대한 위법 부당 행위와 지적사례가 지속적으로 증가함에 따라, 감사를 통해 취약요인을 점검해 올바른 보조금 사용 풍토를 정착시키겠다는 취지다. 시는 감사담당관실과 기획예산과 보조금 관련 주무관으로 특별감사반을 편성해 2017년부터 2018년까지 집행된 축제성 보조금 집행에 대한 철저한 점검과 감사를 통해 부정 수급 및 부정 집행이 확인되면 엄정한 조치를 취할 방침이다. 시는 지난 15일부터 25일까지 10일간의 사전감사를 통해 보조금 실태를 파악한 후, 8월15일까지 세부감사를 진행할 예정이라고 전했다. 축제성 관련 부정수급 유형을 보면 허위·기타 부정한 방법으로 보조금 신청, 사업 실적을 부풀려 보조금을 횡령·편취, 보조금 교부 목적과 다른 용도로 집행, 보조금으로 취득한 재산에 대해 지자체장의 승인없이 임의 처분 등이 해당된다. 시는 불법보조금 근절과 효율적인 점검 및 적극적인 시민관심을 유도하기 위해 \'지방보조금 부정수급 신고센터(☏850-5031)\'를 설치 운영하고 있다. 지방보조금 부정수급 신고 시 직접방문 및 국민신문고(www.epeople.or.kr), 충주시홈페이지(www.chungju.or.kr)를 통해 접수하면 되고, 신고취지와 이유를 기재하고 부정행위와 관련한 증거자료를 제시하면 된다. 단, 익명 신고는 접수치 않는다. 시 관계자는 "이번 자체 점검 및 감사를 통해 축제보조금이 제대로 쓰이는지에 대한 반성과 함께 보조금 집행의 투명성 및 행정의 신뢰성을 확보하는데 최선을 다하겠다"고 말했다. 한편. 시는 감사 및 예산부서 합동으로 컨설팅 위주의 상반기 보조금 특정감사(1월10일~20일)를 실시해 주의 11건, 시정 6건, 권고 1건을 자체 적발하고

## **3. 데이터셋 분할 및 전처리**

In [ ]:
def preprocess(examples):
    inputs = [doc for doc in examples['document']]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['summary'], max_length=MAX_TARGET_LENGTH, truncation=True
        )

    model_inputs['labels'] = labels['input_ids']

    return model_inputs

In [ ]:
raw_datasets = raw_datasets.train_test_split(
    train_size=1-TRAIN_TEST_SPLIT, test_size=TRAIN_TEST_SPLIT
)

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess, batched=True)

Map:   0%|          | 0/2409 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/603 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors='pt')

In [ ]:
train_dataset = tokenized_datasets['train'].to_pandas()                                     # tokenized_datasets['train'] 객체를 pandas dataframe으로 변환
train_dataset = Dataset.from_pandas(train_dataset)                                          # pandas dataframe을 PyTorch Dataset 형식으로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])   # set_format() 함수를 이용해 데이터셋 포맷 변경

test_dataset = tokenized_datasets['test'].to_pandas()
test_dataset = Dataset.from_pandas(test_dataset)
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

generation_dataset = tokenized_datasets["test"].shuffle().select(list(range(200))).to_pandas()
generation_dataset = Dataset.from_pandas(generation_dataset)
generation_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

## **4. 평가 지표 설정**

In [ ]:
rouge = Rouge()

def compute_metrics(pred):
    preds, labels = pred

    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    labels = np.where(labels > 0, labels, tokenizer.pad_token_id)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    score = rouge.get_scores(preds, labels, avg=True)
    
    return {
        "ROUGE-1" : score['rouge-1'],
        "ROUGE-2" : score['rouge-2'],
        "ROUGE-L" : score['rouge-l'],
    }

## **5. 모델 학습**

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',        # 모델 체크포인트 및 로그 저장 경로
    evaluation_strategy='epoch',   # 평가 전략 (epoch 단위로 평가)
    num_train_epochs=MAX_EPOCHS,   # 학습 에포크 수
    save_strategy='epoch',         # 체크포인트 저장 전략 (epoch 단위로 저장)
    logging_steps=100,             # 로그를 출력할 때마다 로깅할 스텝 수
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=model,                       # 사용할 모델
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,       # 훈련 데이터셋
    eval_dataset=generation_dataset,   # 평가 데이터셋
    compute_metrics=compute_metrics,   # 콜백 함수 (메트릭 평가 콜백)
)

# 학습 실행
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,1.625800,1.548461,"{'r': 0.14412287834177215, 'p': 0.4276289682539683, 'f': 0.21018677190918988}","{'r': 0.07135435532348698, 'p': 0.24071427183927185, 'f': 0.1069958051803266}","{'r': 0.13453909622774854, 'p': 0.4011918636918637, 'f': 0.19644050235186794}"
2,1.230700,1.562869,"{'r': 0.14599220334138452, 'p': 0.4263346930846933, 'f': 0.21238330536066438}","{'r': 0.072736493314424, 'p': 0.2409318181818182, 'f': 0.1087340058265818}","{'r': 0.13596831495365594, 'p': 0.39943019480519504, 'f': 0.19817489603393562}"
3,0.909000,1.647295,"{'r': 0.13930814349345894, 'p': 0.4126801393051395, 'f': 0.2037031765006937}","{'r': 0.06842003790812937, 'p': 0.22457099844599845, 'f': 0.1023853896169546}","{'r': 0.13145800150571219, 'p': 0.3898403957153958, 'f': 0.19229013688784627}"
4,0.673900,1.732990,"{'r': 0.14316297652397866, 'p': 0.4133121045621047, 'f': 0.20757367928973924}","{'r': 0.0712660617210071, 'p': 0.23049769674769674, 'f': 0.1060367956650298}","{'r': 0.13466107216087908, 'p': 0.390692640692641, 'f': 0.19556536655270967}"
5,0.542300,1.771937,"{'r': 0.13303447847709726, 'p': 0.39361947774447775, 'f': 0.19405847100999324}","{'r': 0.06438488587682688, 'p': 0.21355463980463985, 'f': 0.09642263453136148}","{'r': 0.12486602760064057, 'p': 0.37150181762681783, 'f': 0.1824516198960947}"


Trainer is attempting to log a value of "{'r': 0.14412287834177215, 'p': 0.4276289682539683, 'f': 0.21018677190918988}" of type <class 'dict'> for key "eval/ROUGE-1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'r': 0.07135435532348698, 'p': 0.24071427183927185, 'f': 0.1069958051803266}" of type <class 'dict'> for key "eval/ROUGE-2" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'r': 0.13453909622774854, 'p': 0.4011918636918637, 'f': 0.19644050235186794}" of type <class 'dict'> for key "eval/ROUGE-L" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'r': 0.14599220334138452, 'p': 0.4263346930846933, 'f': 0.21238330536066438}" of type <class 'dict'> for key "eval/ROUGE-1" as a scal

TrainOutput(global_step=1510, training_loss=1.029279984543655, metrics={'train_runtime': 935.5195, 'train_samples_per_second': 12.875, 'train_steps_per_second': 1.614, 'total_flos': 3635087993671680.0, 'train_loss': 1.029279984543655, 'epoch': 5.0})

## **6. 모델 추론**

In [ ]:
summarizer = pipeline('summarization', model=model.to('cpu'), tokenizer=tokenizer, framework='pt')

summary = summarizer(
              raw_datasets["test"][0]["document"],
              min_length=MIN_TARGET_LENGTH,
              max_length=MAX_TARGET_LENGTH,
          )

In [ ]:
print('원문: ', raw_datasets['test'][0]['document'])
print('-------------------------------------------------------')
print(f"요약문: {[summ['summary_text'] for summ in summary]}")

원문:  강경화 외교부 장관, 마이크 폼페이오 미국 국무장관 등이 참석한 아세안지역안보포럼(ARF) 외무장관 회의가 열린 태국 방콕 도심에서 총 4개의 폭탄이 터져 부상자가 발생했다. 불발탄 2개도 발견됐다. 2일 로이터통신·현지 매체 등은 이날 오전 방콕 도심 최소 3곳에서 폭탄이 터져 4명이 다쳤다고 보도했다. 현지매체 더 타이거(The Thaiger)에 따르면 오전 8시 50분쯤 방콕 중심부인 청농시 BTS 전철역 인근에서 폭발음과 유사한 커다란 굉음이 들렸다. 이 폭발로 전철역 플랫폼 입구의 유리가 깨졌고, 이 역 부근에서만 2명이 다쳤다. 목격자들은 역 바로 앞에 위치한 방콕의 유명한 77층짜리 타워 킹파워 마하나콘 건물 경비실 인근에서도 굉음이 들렸다고 전했다. 경찰은 폭발 이후 전철 입구 두 곳의 출입을 통제했다. 또한 이날 오전 방콕 북부 챙와타나 지역의 정부청사 앞에서도 폭탄이 터졌다. 얼마 지나지 않아 이곳 맞은편에 위치한 정원에서 또다른 폭발이 일었다. 두 건 모두 인명 피해는 없었다. 곧이어 두 지점에서 12km 떨어진 방카피 공항 인근에서 또다른 폭발이 발생했다. 이로 인해 2명의 도로 청소부가 다쳤다. 이후 태국 왕립군사본부 앞 오솔길에서 폭발물 2개가 터지지 않은 채 발견됐다. 이는 앞서 두 건의 폭발이 발생한 정부청사 바로 옆이었다. 이날 터진 폭탄은 탁구공만 한 크기로 이른바 '핑퐁(탁구) 폭탄'이라고 불린다. 현지 경찰은 "타이머로 작동된 것으로 보인다"고 밝혔다. 태국 정부 대변인은 쁘라윳 짠오차 총리가 폭탄 사건을 보고받고 즉시 해당 사건에 대한 조사를 지시했다고 전했다. 이날 방콕에선 동남아시아 국가들과 한국·미국·일본·중국·러시아 등 각국 외교 수장들이 참여한 ARF 회의가 열렸다. 행사에는 강경화 외교부 장관과 마이크 폼페이오 미 국무장관, 고노 다로 일본 외무상 등이 참석했다.
-------------------------------------------------------
요약문: ['아세안지역안보포럼(ARF) 외

## **[참고] 디코딩 전략에 따른 요약문 생성**

In [ ]:
summary_decoding = summarizer(
              raw_datasets['test'][0]['document'],
              min_length=MIN_TARGET_LENGTH,
              max_length=MAX_TARGET_LENGTH,
              do_sample=True,
              top_k=50,  
              top_p=0.92,
              temperature=2.5,
              num_return_sequences=4,
          )

In [ ]:
print('원문: ', raw_datasets['test'][0]['document'])
print('-------------------------------------------------------')
print(f"요약문: {[summ['summary_text'] for summ in summary_decoding]}")

원문:  강경화 외교부 장관, 마이크 폼페이오 미국 국무장관 등이 참석한 아세안지역안보포럼(ARF) 외무장관 회의가 열린 태국 방콕 도심에서 총 4개의 폭탄이 터져 부상자가 발생했다. 불발탄 2개도 발견됐다. 2일 로이터통신·현지 매체 등은 이날 오전 방콕 도심 최소 3곳에서 폭탄이 터져 4명이 다쳤다고 보도했다. 현지매체 더 타이거(The Thaiger)에 따르면 오전 8시 50분쯤 방콕 중심부인 청농시 BTS 전철역 인근에서 폭발음과 유사한 커다란 굉음이 들렸다. 이 폭발로 전철역 플랫폼 입구의 유리가 깨졌고, 이 역 부근에서만 2명이 다쳤다. 목격자들은 역 바로 앞에 위치한 방콕의 유명한 77층짜리 타워 킹파워 마하나콘 건물 경비실 인근에서도 굉음이 들렸다고 전했다. 경찰은 폭발 이후 전철 입구 두 곳의 출입을 통제했다. 또한 이날 오전 방콕 북부 챙와타나 지역의 정부청사 앞에서도 폭탄이 터졌다. 얼마 지나지 않아 이곳 맞은편에 위치한 정원에서 또다른 폭발이 일었다. 두 건 모두 인명 피해는 없었다. 곧이어 두 지점에서 12km 떨어진 방카피 공항 인근에서 또다른 폭발이 발생했다. 이로 인해 2명의 도로 청소부가 다쳤다. 이후 태국 왕립군사본부 앞 오솔길에서 폭발물 2개가 터지지 않은 채 발견됐다. 이는 앞서 두 건의 폭발이 발생한 정부청사 바로 옆이었다. 이날 터진 폭탄은 탁구공만 한 크기로 이른바 '핑퐁(탁구) 폭탄'이라고 불린다. 현지 경찰은 "타이머로 작동된 것으로 보인다"고 밝혔다. 태국 정부 대변인은 쁘라윳 짠오차 총리가 폭탄 사건을 보고받고 즉시 해당 사건에 대한 조사를 지시했다고 전했다. 이날 방콕에선 동남아시아 국가들과 한국·미국·일본·중국·러시아 등 각국 외교 수장들이 참여한 ARF 회의가 열렸다. 행사에는 강경화 외교부 장관과 마이크 폼페이오 미 국무장관, 고노 다로 일본 외무상 등이 참석했다.
-------------------------------------------------------
요약문: ["동남아 3개국가들과 미국, 중